In [1]:
from basyx.aas.model import Identifier

In [3]:
#!/usr/bin/env python3
# 이 작업은 Creative Commons CCZero 1.0 Universal License에 따라 라이선스가 부여됩니다.
# 자세한 내용은 http://creativecommons.org/publicdomain/zero/1.0/을 참조하세요.
"""
AssetInformation 객체와 하위 모델을 포함하는 간단한 Asset Administration Shell을 만드는 방법에 대한 튜토리얼 
"""

# Import all Eclipse BaSyx Python SDK classes from model package
from basyx.aas import model

# 이 튜토리얼에서는 Asset Administration Shell(AAS)과 그 안에 있는 모든 필수 객체를 만드는 방법에 대한 단계별 가이드를 제공합니다.
# 먼저 AAS를 만들려는 AssetInformation 객체가 필요합니다. 그 후에
# Asset Administration Shell을 만들 수 있습니다. 그런 다음 AAS에 하위 모델을 추가할 수 있습니다. 하위 모델은
# SubmodelElements를 포함할 수 있습니다.
#
# 단계별 가이드:
# 1단계: AssetInformation 객체를 포함하는 간단한 자산 관리 셸 만들기
# 2단계: 간단한 하위 모델 만들기
# 3단계: 간단한 속성을 만들고 하위 모델에 추가


##################################################################
# Step 1: AssetInformation 객체를 포함하는 간단한 자산 관리 셸 만들기
##################################################################
# Step 1.1: create the AssetInformation object
asset_information = model.AssetInformation(
    asset_kind=model.AssetKind.INSTANCE,
    global_asset_id='http://acplt.org/Simple_Asset'
)

# step 1.2: create the Asset Administration Shell
identifier = 'https://acplt.org/Simple_AAS'
aas = model.AssetAdministrationShell(
    id_=identifier,  # set identifier
    asset_information=asset_information
)


#############################################################
# Step 2: SubmodelElements 없이 간단한 하위 모델 만들기 #
#############################################################

# Step 2.1: create the Submodel object
identifier = 'https://acplt.org/Simple_Submodel'
submodel = model.Submodel(
    id_=identifier
)

# Step 2.2: 해당 하위 모델에 대한 참조를 생성하고 자산 관리 셸의 `하위 모델` 세트에 추가합니다.
aas.submodel.add(model.ModelReference.from_referable(submodel))


# ===============================================================
# 대안: 1단계와 2단계는 대안적으로 한 단계로 수행될 수 있습니다.
# 이 버전에서 Submodel 참조는 Asset Administration Shell의 생성자로 전달됩니다.
submodel = model.Submodel(
    id_='https://acplt.org/Simple_Submodel'
)
aas = model.AssetAdministrationShell(
    id_='https://acplt.org/Simple_AAS',
    asset_information=asset_information,
    submodel={model.ModelReference.from_referable(submodel)}
)


#######################################################
# Step 3: 간단한 속성을 생성하여 하위 모델에 추가합니다.
#######################################################

# Step 3.1: Property의 Semantic description에 대한 글로벌 참조를 만듭니다.
# 글로벌 참조는 Semantic description이 저장된 주소를 가리키는 하나의 키로 구성됩니다.
semantic_reference = model.ExternalReference(
    (model.Key(
        type_=model.KeyTypes.GLOBAL_REFERENCE,
        value='http://acplt.org/Properties/SimpleProperty'
    ),)
)

# Step 3.2: create the simple Property
property_ = model.Property(
    id_short='ExampleProperty',  # Submodel 네임스페이스 내의 요소 식별 문자열
    value_type=model.datatypes.String,  # Data type of the value
    value='exampleValue',  # Value of the Property
    semantic_id=semantic_reference  # set the semantic reference
)

# Step 3.3: add the Property to the Submodel
submodel.submodel_element.add(property_)


# =====================================================================
# 대안: 2단계와 3단계도 단일 문장으로 결합할 수 있습니다.
# 다시 말하지만, 나중에 추가하는 대신 Property를 Submodel의 생성자에 전달합니다.
submodel = model.Submodel(
    id_='https://acplt.org/Simple_Submodel',
    submodel_element={
        model.Property(
            id_short='ExampleProperty',
            value_type=model.datatypes.String,
            value='exampleValue',
            semantic_id=model.ExternalReference(
                (model.Key(
                    type_=model.KeyTypes.GLOBAL_REFERENCE,
                    value='http://acplt.org/Properties/SimpleProperty'
                ),)
            )
        )
    }
)

In [1]:
#!/usr/bin/env python3
# This work is licensed under a Creative Commons CCZero 1.0 Universal License.
# See http://creativecommons.org/publicdomain/zero/1.0/ for more information.
"""
JSON 및 XML 파일에 자산 관리 셸, 하위 모델 및 자산을 직렬화 및 역직렬화하는 방법에 대한 튜토리얼입니다.
"""

import json

from basyx.aas import model
import basyx.aas.adapter.json
import basyx.aas.adapter.xml

# '자산 관리 셸의 세부 정보'는 AAS 데이터에 대한 여러 가지 공식 직렬화 형식을 지정합니다. 이
# 튜토리얼에서는 Eclipse BaSyx Python 라이브러리를 사용하여 AAS 객체를 JSON 또는 XML로 직렬화하고
# 표준화된 형식에 따라 JSON/XML 파일을 만드는 방법을 보여줍니다. 또한 이러한 파일을 구문 분석하여
# AAS 객체를 Python 객체로 복원하는 방법도 보여줍니다.
#
# Step-by-Step Guide:
# 1단계: Submodel 및 Asset Administration Shell 객체 생성
# 2단계: 단일 객체를 JSON으로 직렬화
# 3단계: JSON에서 단일 객체 또는 사용자 정의 데이터 구조 구문 분석
# 4단계: 여러 식별 가능한 객체를 (표준 준수) JSON/XML 파일에 쓰기
# 5단계: JSON/XML 파일에서 직렬화된 aas 객체 읽기

####################################################################
# Step 1: Submodel 및 AAS 객체 생성 #
####################################################################

# For more details, take a look at `tutorial_create_simple_aas.py`

submodel = model.Submodel(
    id_='https://acplt.org/Simple_Submodel',
    submodel_element={
        model.Property(
            id_short='ExampleProperty',
            value_type=basyx.aas.model.datatypes.String,
            value='exampleValue',
            semantic_id=model.ExternalReference((model.Key(
                    type_=model.KeyTypes.GLOBAL_REFERENCE,
                    value='http://acplt.org/Properties/SimpleProperty'
                ),)
            )
        )}
)
aashell = model.AssetAdministrationShell(
    id_='https://acplt.org/Simple_AAS',
    asset_information=model.AssetInformation(global_asset_id="test"),
    submodel={model.ModelReference.from_referable(submodel)}
)


##############################################
# Step 2: 단일 객체를 JSON으로 직렬화 #
##############################################

# 데이터를 직렬화하기 전에 최신 상태인지 확인해야 합니다. 이는 이 튜토리얼의 정적 AAS 객체에는 중요하지 않지만
# 동적 데이터를 처리할 때는 중요할 수 있습니다.
# 해당 주제에 대한 자세한 내용은 `tutorial_dynamic_model.py`를 참조하세요.
aashell.update()

# '자산 관리 셸의 세부 사항', 5.5장, Python의 내장 JSON 라이브러리 사용.
#`json.dump()` 및 `json.dumps()` 메서드에 제공되면 이러한 메서드는 덤프된 데이터 구조 내에서 AAS 객체를 올바르게 처리할 수 있습니다.

aashell_json_string = json.dumps(aashell, cls=basyx.aas.adapter.json.AASToJsonEncoder)

property_json_string = json.dumps(submodel.submodel_element.get_object_by_attribute("id_short", 'ExampleProperty'),
                                  cls=basyx.aas.adapter.json.AASToJsonEncoder)

# 이 기술을 사용하면 중첩된 AAS 객체로 Python dict 및 list 데이터 구조를 직렬화할 수도 있습니다.
json_string = json.dumps({'the_submodel': submodel,
                          'the_aas': aashell
                          },
                         cls=basyx.aas.adapter.json.AASToJsonEncoder)


######################################################################
# Step 3: JSON에서 단일 객체 또는 사용자 정의 데이터 구조 구문 분석 #
######################################################################

# JSON 역직렬화는 JSON 직렬화와 비슷한 방식으로 작동합니다.
# `aas.adapter.json` 모듈은 `AASFromJSONDecoder`라는 JSONDecoder 클래스를 제공하며, 이 클래스는 `json.load()`또는 `json.loads()`에 전달되어 
# JSON 데이터에 포함된 AAS 객체가 간단한 Python 사전 대신 BaSyx Python SDK 객체 표현으로 변환되도록 합니다.
submodel_and_aas = json.loads(json_string, cls=basyx.aas.adapter.json.AASFromJsonDecoder)

# 또는 `StrictAASFromJsonDecoder`를 사용할 수 있는데, 이는 정확히 같은 방식으로 작동하지만 형식 사양을 더 엄격하게 적용합니다.
# `AASFromJSONDecoder`는 깨진 객체를 간단히 건너뛰고 로그 메시지를 발행하여 일부 의미 오류를 허용하지만, `StrictAASFromJsonDecoder`는 이러한 경우 예외를 발생시킵니다.


#########################################################################################
# Step 4: (표준 호환) JSON/XML 파일에 여러 식별 가능한 객체 쓰기 #
#########################################################################################

# step 4.1: 직렬화할 객체를 포함하는 ObjectStore 생성
# For more information, take a look into `tutorial_storage.py`
obj_store: model.DictObjectStore[model.Identifiable] = model.DictObjectStore()
obj_store.add(submodel)
obj_store.add(aashell)

# step 4.2: 다시 한번, 데이터가 최신 상태인지 확인하세요.
submodel.update()
aashell.update()

# step 4.3: ObjectStore의 내용을 JSON 파일에 쓰기
basyx.aas.adapter.json.write_aas_json_file('data.json', obj_store)

# 추가 키워드 인수인 `indent=4`를 `write_aas_json_file()`에 전달하면 JSON 파일의 형식을 사람이 더 읽기 쉬운(하지만 훨씬 더 많은 공간을 차지하는) 방식으로 지정할 수 있습니다.

# step 4.4: ObjectStore의 내용을 XML 파일에 쓰기
basyx.aas.adapter.xml.write_aas_xml_file('data.xml', obj_store)


##################################################################
# Step 5: JSON/XML 파일에서 직렬화된 AAS 개체 읽기 #
##################################################################

# step 5.1: JSON 파일의 내용을 ObjectStore로 읽기
json_file_data = basyx.aas.adapter.json.read_aas_json_file('data.json')

# `read_aas_json_file()`에 `failsafe=False` 인수를 전달하면 `StrictAASFromJsonDecoder`로 전환할 수 있습니다.
# (더 엄격한 오류 보고를 위해 3단계 참조)

# step 5.2: XML 파일의 내용을 ObjectStore로 읽기
xml_file_data = basyx.aas.adapter.xml.read_aas_xml_file('data.xml')

# 다시 말해, `failsafe=False`를 사용하면 파서에서 더 엄격한 오류 보고를 켤 수 있습니다.

# step 5.3: ObjectStore에서 객체 검색
# For more information on the availiable techniques, see `tutorial_storage.py`.
submodel_from_xml = xml_file_data.get_identifiable('https://acplt.org/Simple_Submodel')
assert isinstance(submodel_from_xml, model.Submodel)

In [5]:
#!/usr/bin/env python3
# This work is licensed under a Creative Commons CCZero 1.0 Universal License.
# See http://creativecommons.org/publicdomain/zero/1.0/ for more information.
"""
ObjectStore에 자산 관리 셸, 하위 모델 및 자산을 저장하고 이를 사용하여 ID로 이러한 객체를 가져오고 참조를 확인하는 방법에 대한 튜토리얼입니다.

"""

# 더 많은 식별 가능한 AAS 개체(AssetAdministrationShells, Assets, Submodels,ConceptDescriptions)를 
# 관리하기 위해 BaSyx Python SDK는 'ObjectStore' 기능을 제공합니다. 
# 이 튜토리얼에서는 ObjectStore의 기본 기능과 사용 방법을 보여줍니다.
# 여기에는 `AssetAdministrationShell.submodel` 세트 등에서 참조되는 
# 하위 모델 객체를 쉽게 가져오는 데 사용할 수 있는 참조 객체의 내장 `resolve()` 메서드 사용이 포함됩니다.

# Step-by-Step Guide:
# Step 1: AssetInformation, 하위 모델 및 자산 관리 셸 객체 생성
# Step 2: 보다 쉬운 처리를 위해 ObjectStore에 데이터 저장
# Step 3: 식별자로 저장소에서 객체 검색
# Step 4: ObjectStore를 사용하여 참조 해결


from basyx.aas import model
from basyx.aas.model import AssetInformation, AssetAdministrationShell, Submodel


######################################################################################
# Step 1: AssetInformation, 하위 모델 및 자산 관리 셸 객체 생성 #
######################################################################################

# For more details, take a look at `tutorial_create_simple_aas.py`

asset_information = AssetInformation(
    asset_kind=model.AssetKind.INSTANCE,
    global_asset_id='http://acplt.org/Simple_Asset'
)

prop = model.Property(
    id_short='ExampleProperty',
    value_type=model.datatypes.String,
    value='exampleValue',
    semantic_id=model.ExternalReference(
        (model.Key(
            type_=model.KeyTypes.GLOBAL_REFERENCE,
            value='http://acplt.org/Properties/SimpleProperty'
        ),)
    )
)
submodel = Submodel(
    id_='https://acplt.org/Simple_Submodel',
    submodel_element={prop}
)
aas = AssetAdministrationShell(
    id_='https://acplt.org/Simple_AAS',
    asset_information=asset_information,
    submodel={model.ModelReference.from_referable(submodel)}
)


##################################################################
# Step 2: 보다 쉬운 처리를 위해 ObjectStore에 데이터 저장 #
##################################################################

# Step 2.1: 식별 가능한 객체에 대한 ObjectStore 생성
#
# 이 튜토리얼에서는 간단한 메모리 내 저장소인 `DictObjectStore`를 사용합니다. 이는 단지 dict를 사용하여 Python 객체를 추적합니다.
# 많은 수의 개체를 관리해야 하거나 개체를 영구 메모리(예: 하드 디스크)에 보관해야 하는 경우 이는 적합한 솔루션이 아닐 수 있습니다.
# 이 경우 `aas.backends.couchdb`에서 `CouchDBObjectStore`를 선택하여 CouchDB 데이터베이스 서버를 영구 저장소로 사용할 수 있습니다.
# 두 ObjectStore 구현 모두 동일한 인터페이스를 제공합니다.
# 또한 CouchDBObjectStores를 사용하면 백엔드 및 update()/commit() 메커니즘을 통해 로컬 객체를 데이터베이스와 동기화할 수 있습니다.
# 자세한 내용은 `tutorial_backend_couchdb.py`를 참조하세요.

obj_store: model.DictObjectStore[model.Identifiable] = model.DictObjectStore()

# step 2.2: add submodel and asset administration shell to store
obj_store.add(submodel)
obj_store.add(aas)


#################################################################
# Step 3: 식별자로 저장소에서 개체 검색 #
#################################################################

tmp_submodel = obj_store.get_identifiable(
    'https://acplt.org/Simple_Submodel')

assert submodel is tmp_submodel


########################################################
# Step 4: ObjectStore를 사용하여 참조 확인 #
########################################################

# `aas` 객체에는 이미 하위 모델에 대한 참조가 포함되어 있습니다.
# 각 하위 모델 참조를 분석하여 AAS가 참조하는 모든 하위 모델 목록을 생성해 보겠습니다.

submodels = [reference.resolve(obj_store)
             for reference in aas.submodel]

# 이 목록의 첫 번째(그리고 유일한) 요소는 예제 하위 모델이어야 합니다.:
assert submodel is submodels[0]

# Now, let's manually create a reference to the Property within the submodel. The reference uses two keys, the first one
# identifying the submodel by its id, the second one resolving to the Property within the submodel by its
# idShort.
property_reference = model.ModelReference(
    (model.Key(
        type_=model.KeyTypes.SUBMODEL,
        value='https://acplt.org/Simple_Submodel'),
     model.Key(
         type_=model.KeyTypes.PROPERTY,
         value='ExampleProperty'),
     ),
    type_=model.Property
)

# Now, we can resolve this new reference.
# The `resolve()` method will fetch the Submodel object from the ObjectStore, traverse down to the included Property
# object and return this object.
tmp_property = property_reference.resolve(obj_store)
assert prop is tmp_property

In [11]:
from basyx.aas import model
from basyx.aas.model.datatypes import DataTypeDefXsd

# AAS 및 자산 생성
asset = model.Asset(
    id_short="ExampleAsset",
    identification=model.Identifier(id_="example_asset_123", id_type=model.IdentifierType.IRI),
    kind=model.AssetKind.INSTANCE
)

aas = model.AssetAdministrationShell(
    id_short="ExampleAAS",
    identification=model.Identifier(id_="example_aas_123", id_type=model.IdentifierType.IRI),
    asset_information=model.AssetInformation(asset_kind=model.AssetKind.INSTANCE, global_asset_id=model.Identifier(id_="example_asset_123", id_type=model.IdentifierType.IRI))
)

# 서브모델 생성
submodel = model.Submodel(
    id_short="ExampleSubmodel",
    identification=model.Identifier(id_="example_submodel_123", id_type=model.IdentifierType.IRI),
    submodel_elements=[
        model.Property(
            id_short="ExampleProperty",
            value_type=DataTypeDefXsd.FLOAT,
            value=42.0
        )
    ]
)

# AAS에 서브모델 추가
aas.submodels.append(submodel)


ImportError: cannot import name 'DataTypeDefXsd' from 'basyx.aas.model.datatypes' (c:\ProgramData\Anaconda3\envs\aas\Lib\site-packages\basyx\aas\model\datatypes.py)